In [ ]:
import time
import numpy as np
from numba import njit, prange, cuda, types, float32, float64, int32
import matplotlib.pyplot as plt
import os

In [ ]:
@njit
def testpoint(rp , ri, max_iter):
    crp = rp
    cri = ri
    count = 0
    for i in range(0, max_iter):
        temp = crp * crp - cri * cri + rp
        cri = crp * cri * 2.0 + ri
        crp = temp
         
        if (crp * crp + cri * cri) > 4.0:
            break
            
    return i

@njit
def mandelbrot(m : int, npoints, max_iter, eps):
    for i in range(0, npoints):
        for j in range(0, npoints):
            rp = -2.0 + 3.0 * float(i) / float(npoints) + eps
            ri = -1.5 + 3.0 * float(j) / float(npoints) + eps
            
            m[i,j] = testpoint(rp, ri, max_iter)

In [ ]:
npoints = 2000
max_iter = 100
eps = 1.0e-6

m = np.zeros(shape=(npoints,npoints), dtype=int)

start=time.time()
mandelbrot(m, npoints, max_iter, eps)
end=time.time()

n = np.count_nonzero(m < max_iter-1) # convert to bool

print('Elapsed time: ', end-start)

area = 3.0 * 3.0 * float(npoints*npoints-n) / float(npoints * npoints)
error = area / npoints

print('area: ', area, n)
print('error: ', error)

In [ ]:
ax = plt.axes()
ax.set_aspect('equal')
plt.imshow(m)

In [ ]:
@cuda.jit(device=True, inline=True)
def testpoint(rp , ri, max_iter):
    crp = rp
    cri = ri
    count = 0
    for i in range(0, max_iter):
        temp = crp * crp - cri * cri + rp
        cri = crp * cri * 2.0 + ri
        crp = temp
         
        if (crp * crp + cri * cri) > 4.0:
            break
            
    return i

@cuda.jit
def mandelbrot(m, npoints, max_iter, eps):
    i=cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    j=cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    
    rp = -2.0 + 3.0 * float(i) / float(npoints) + eps
    ri = -1.5 + 3.0 * float(j) / float(npoints) + eps

    if i < npoints and j < npoints:
        m[i,j] = testpoint(rp, ri, max_iter)

In [ ]:
npoints = 2000 # 16000
max_iter = 100 # 2000
eps = 1.0e-6

m = np.zeros(shape=(npoints,npoints), dtype=np.int32)

threadsperblock = (16,16)
blockspergrid_x = int(np.ceil(npoints / threadsperblock[0]))
blockspergrid_y = int(np.ceil(npoints / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

start=time.time()
mandelbrot[blockspergrid, threadsperblock](m, npoints, max_iter, eps)
end=time.time()

print('Elapsed time: ', end-start)

n = np.count_nonzero(m < max_iter-1) # convert to bool

area = 3.0 * 3.0 * float(npoints*npoints-n) / float(npoints * npoints)
error = area / npoints

print('area: ', area, n)
print('error: ', error)

In [ ]:
ax = plt.axes()
ax.set_aspect('equal')
plt.imshow(m)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(m[int(npoints/2.5):int(npoints/2.3),int(npoints/2.2):int(npoints/2.0)])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(m[int(npoints/2.44):int(npoints/2.42),int(npoints/2.14):int(npoints/2.12)])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(m[int(npoints/2.85):int(npoints/2.7),int(npoints/2.45):int(npoints/2.35)])